In [ ]:
class MyLogReg():
  def __init__(self, n_iter = 10, learning_rate = 0.1, weights = None, metric = None,reg = None, l1_coef = None, l2_coef = None, **kwargs):
    self.n_iter = n_iter
    self.learning_rate = learning_rate
    self.weights = weights
    self.metric = metric
    self.reg = reg
    self.l1_coef = l1_coef
    self.l2_coef = l2_coef
    self.params = kwargs

  def __str__(self):
    base_str = f'MyLogReg class: n_iter={self.n_iter}, learning_rate={self.learning_rate}'
    if self.params:
      param_str = ', '.join([f"{key}={value}" for key, value in self.params.items()])
      return f"{base_str}, {param_str}"
    return base_str

  def calculate_metric(self, y_true, y_pred):
    if self.metric == 'accuracy':
      return np.mean(y_true == (y_pred > 0.5).astype(int))
    elif self.metric == 'precision':
      true_positive = np.sum((y_true == 1) & (y_pred > 0.5))
      false_positive = np.sum((y_true == 0) & (y_pred > 0.5))
      return true_positive / (true_positive + false_positive)
    elif self.metric == 'recall':
      true_positive = np.sum((y_true == 1) & (y_pred > 0.5))
      false_negative = np.sum((y_true == 1) & (y_pred <= 0.5))
      return true_positive / (true_positive + false_negative)
    elif self.metric == 'f1':
      true_positive = np.sum((y_true == 1) & (y_pred > 0.5))
      false_positive = np.sum((y_true == 0) & (y_pred > 0.5))
      false_negative = np.sum((y_true == 1) & (y_pred <= 0.5))
      precision = true_positive / (true_positive + false_positive)
      recall = true_positive / (true_positive + false_negative)
      return 2 * (precision * recall) / (precision + recall)
    elif self.metric == 'roc_auc':
      y_pred = y_pred.round(decimals = 10)
      s = 0
      P = (y_true == 1).sum()
      N = (y_true == 0).sum()
      y_true_values = y_true.values
      y_pred_values = y_pred.values
      for i in range(len(y_true)):
        for j in range(len(y_true)):
          if y_true_values[i] >= y_true_values[j]:
            I = 0
          else:
            I = 1

          if y_pred_values[i] > y_pred_values[j]:
            I1 = 0
          elif y_pred_values[i] == y_pred_values[j]:
            I1 = 0.5
          else:
            I1 = 1

          s = s + I * I1
      return s / (P * N)
    else:
      return None


  def fit(self, X, y, verbose = False):
    eps = 1e-15
    X.insert(0, 'Ones', 1)
    self.weights = np.ones(X.shape[1])

    for i in range(self.n_iter):
      y_pred = 1 / (1 + np.exp(-X.dot(self.weights)))
      LogLoss = -np.mean(y * np.log(y_pred + eps) + (1 - y) * np.log(1 - y_pred + eps))
      gradient = 1 / len(y) * (y_pred - y).dot(X)

      if self.reg == 'l1':
        LogLoss = LogLoss + self.l1_coef * np.sum(np.abs(self.weights))
        gradient = gradient + self.l1_coef * np.sign(self.weights)
      elif self.reg == 'l2':
        LogLoss = LogLoss + self.l2_coef * np.sum(np.square(self.weights))
        gradient = gradient + self.l2_coef * 2 * self.weights
      elif self.reg == 'elasticnet':
        LogLoss = LogLoss + self.l1_coef * np.sum(np.abs(self.weights)) + self.l2_coef * np.sum(np.square(self.weights))
        gradient = gradient + self.l1_coef * np.sign(self.weights) + self.l2_coef * 2 * self.weights

      if isinstance(self.learning_rate, (int, float)):
        self.weights -= self.learning_rate * gradient
      else:
        self.weights -= self.learning_rate(i + 1) * gradient

      if self.metric:
        if verbose and (i + 1) % verbose == 0:
          self.metric_value = self.calculate_metric(y, y_pred)
          print(f'{i + 1} / LogLoss : {LogLoss} / {self.metric} : {self.metric_value}')
      else:
        if verbose and (i + 1) % verbose == 0:
          print(f'{i + 1} / LogLoss : {LogLoss}')

  def get_best_score(self):
    y_pred = 1 / (1 + np.exp(-X.dot(self.weights)))
    return self.calculate_metric(y, y_pred)

  def get_coef(self):
    return np.array(self.weights)[1:]

  def predict(self, X):
    X.insert(0, 'Ones', 1)
    y_pred = 1 / (1 + np.exp(-X.dot(self.weights)))
    y_pred = np.array([1 if i > 0.5 else 0 for i in y_pred])
    return y_pred

  def predict_proba(self, X):
    y_pred = 1 / (1 + np.exp(-X.dot(self.weights)))
    return y_pred